In [11]:
from google.colab import drive
# drive.mount("/content/drive", force_remount=True)
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [12]:
import os
import nibabel as nib
import numpy as np

!pip install SimpleITK   # not installed by default
import SimpleITK as sitk

from scipy.ndimage.interpolation import rotate

In [18]:
# need to set these variables yourself
nii_folder = '/content/drive/My Drive/Hematoma Expansion Cohorts/Final Images/dataset/SPOTLIGHT/images/' # folder containing the nii files
dicom_folders = '/content/drive/My Drive/Hematoma Expansion Cohorts/Study Cohorts/SPOTLIGHT/SPOTLIGHT - DICOM/' # folder containing the DICOM series folders

In [14]:
def get_voxel_spacing_from_dicom_serie(dicom_serie_directory):

  dcm_files = [name for name in os.listdir(dicom_serie_directory)]

  # The following code was taken from -> https://discourse.itk.org/t/reading-dicom-series-with-simpleitk-and-filesort/1577
  # A file name that belongs to the series we want to read
  file_name = os.path.join(dicom_serie_directory, dcm_files[0])
  data_directory = dicom_serie_directory

  # Read the file's meta-information without reading bulk pixel data
  file_reader = sitk.ImageFileReader()
  file_reader.SetFileName(file_name)
  file_reader.ReadImageInformation()

  # Get the sorted file names, opens all files in the directory and reads the meta-information
  # without reading the bulk pixel data
  series_ID = file_reader.GetMetaData('0020|000e')
  sorted_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(data_directory, series_ID)

  # Read the bulk pixel data
  img_dcm = sitk.ReadImage(sorted_file_names)

  # get the voxel spacing
  voxel_spacing = img_dcm.GetSpacing()
  voxel_spacing = list(voxel_spacing)
  print('sitk voxel spacing:', voxel_spacing)
  return voxel_spacing

def get_dicom_serie_directory_from_nii_filename(filename):
    dataset_name = filename.split('-')[0]
    site_ID = filename.split('-')[1]
    if site_ID[0] == str(0):
      site_ID = site_ID[-1]
    if int(site_ID) < 10:
      img_ID = '0' + site_ID + '-' + filename.split('-')[2][:3]
    elif int(site_ID) >= 10:
      img_ID = site_ID + '-' + filename.split('-')[2][:3] 
    print(img_ID)
    dicom_serie_dir = os.path.join(dicom_folders, 'Site ' + site_ID, dataset_name + '_' + img_ID, 'series')
    print(dicom_serie_dir)
    return dicom_serie_dir

In [19]:
def fix_nii_voxel_spacing(nii_folder, dicom_folders):

  print(os.path.exists(nii_folder))

  nii_files = [name for name in os.listdir(nii_folder) if '.nii.gz' in name]
  n_affine_fixed = 0
  for name in nii_files:
    filepath = os.path.join(nii_folder, name)
    img = nib.load(filepath)
    # print(name)

    if np.allclose(img.affine, np.eye(4)):  # if true then voxel spacing data needs to be fetched from original dicom series
      # get the right dicom serie folder name and then extract voxel_spacing vector from the serie
      voxel_spacing = get_voxel_spacing_from_dicom_serie(get_dicom_serie_directory_from_nii_filename(name))

      affine_matrix = img.affine
      print('affine matrix before ->', affine_matrix)
      for i in range(len(voxel_spacing)):
        affine_matrix[i,i] = voxel_spacing[i]
      print('affine matrix after ->', affine_matrix)

      # overwrite the original file with the fixed affine_matrix
      new_img = nib.Nifti1Image(img.get_fdata(), affine_matrix, img.header)
      new_img.to_filename(filepath)
      n_affine_fixed += 1

  print('number of files with affine matrix that were fixed:', n_affine_fixed)

In [20]:
nii_files = [name for name in os.listdir(nii_folder) if '.nii.gz' in name]
n_affine_fixed = 0
for name in nii_files:
  filepath = os.path.join(nii_folder, name)
  img = nib.load(filepath)
  if np.allclose(img.affine, np.eye(4)):  # if true then voxel spacing data needs to be fetched from original dicom series
      # get the right dicom serie folder name and then extract voxel_spacing vector from the serie
      print(name)
      n_affine_fixed += 1

print('number of files with affine matrix that need fixing:', n_affine_fixed)

number of files with affine matrix that need fixing: 0


In [17]:
nii_folders = [nii_folder, os.path.join(os.path.dirname(nii_folder), 'imagesTr'), os.path.join(os.path.dirname(nii_folder), 'imagesTs')]

for nii_folder in nii_folders:
  fix_nii_voxel_spacing(nii_folder, dicom_folders)

True
04-002
/content/drive/My Drive/Hematoma Expansion Cohorts/Study Cohorts/PREDICT/PREDICT - DICOM/Site 4/PREDICT_04-002/series
sitk voxel spacing: [0.390625, 0.390625, 4.7740020751953125]
affine matrix before -> [[ 1.  0.  0. -0.]
 [ 0.  1.  0. -0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
affine matrix after -> [[ 0.390625    0.          0.         -0.        ]
 [ 0.          0.390625    0.         -0.        ]
 [ 0.          0.          4.77400208  0.        ]
 [ 0.          0.          0.          1.        ]]
04-004
/content/drive/My Drive/Hematoma Expansion Cohorts/Study Cohorts/PREDICT/PREDICT - DICOM/Site 4/PREDICT_04-004/series
sitk voxel spacing: [0.42773438, 0.42773438, 1.5]
affine matrix before -> [[ 1.  0.  0. -0.]
 [ 0.  1.  0. -0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
affine matrix after -> [[ 0.42773438  0.          0.         -0.        ]
 [ 0.          0.42773438  0.         -0.        ]
 [ 0.          0.          1.5         0.        ]
 [ 0.          0.          0

FileNotFoundError: ignored